# How to modify an existing slice

This notebook outlines the process through which users can adjust an existing slice by either incorporating or eliminating nodes, components, and networks.

## Import the FABlib Library


In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

## Create the Experiment Slice

- Initialize a slice comprising two nodes: Node1 located at Site1 and Node2 at Site2, linked via a layer2 wide area network.
- Incorporate Node3 at Site3 and establish a layer2 wide area network connection between Node1 and Node3.

![Initial Topology](./initial-slice.png)

In [ ]:
slice_name = 'MySlice-modify-add-node-network'
[site1, site2, site3] = fablib.get_random_sites(count=3)

print(f"Sites: {site1}, {site2}, {site3}")

node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'
node4_name = 'Node4'

network1_name='net1'
network2_name='net2'
network3_name='net3'

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Networks
net1 = slice.add_l2network(name=network1_name, subnet=IPv4Network("192.168.1.0/24"))


# Node1
node1 = slice.add_node(name=node1_name, site=site1)
iface1 = node1.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
iface1.set_mode('auto')
net1.add_interface(iface1)


# Node2
node2 = slice.add_node(name=node2_name, site=site2)
iface2 = node2.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
iface2.set_mode('auto')
net1.add_interface(iface2)


#Submit Slice Request
slice.submit();

## Verify connectivity between Node1 and Node2

In [ ]:
slice = fablib.get_slice(slice_name)
node1 = slice.get_node(name=node1_name)
node2 = slice.get_node(name=node2_name)

In [ ]:
node2_addr = node2.get_interface(network_name=network1_name).get_ip_addr()

stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')

## Modify the slice


NOTE: Ensure to retrieve the most recent slice topology prior to initiating any update requests using `slice = fablib.get_slice(slice_name)`.

- Integrate Node3 at site3 equipped with a NIC.
- Add a NIC to Node1.
- Establish a connection between Node1 and Node3.
- Integrate Node4 at site1 equipped with a NIC.
- Add a NIC to Node1.
- Establish a connection between Node1 and Node4.

![Updated Topology](./modify-add.png)

In [ ]:
## NOTE: Always get the latest slice topolgy before requesting any updates
slice = fablib.get_slice(slice_name)

# Add a Layer2 Network
net2 = slice.add_l2network(name=network2_name, subnet=IPv4Network("192.168.2.0/24"))


# Add Node3
node3 = slice.add_node(name=node3_name, site=site2)
iface3 = node3.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
iface3.set_mode('auto')
net2.add_interface(iface3)

# Add NIC to Node1 and add connect to Node3
node1 = slice.get_node(name=node1_name)
iface4 = node1.add_component(model='NIC_Basic', name='nic2').get_interfaces()[0]
iface4.set_mode('auto')
net2.add_interface(iface4)


# Add a Layer2 Network
net3 = slice.add_l2network(name=network3_name, subnet=IPv4Network("192.168.3.0/24"))

# Add Node4
node4 = slice.add_node(name=node4_name, site=site1)
iface5 = node4.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
iface5.set_mode('auto')
net3.add_interface(iface5)

# Add NIC to Node1 and add connect to Node3
node1 = slice.get_node(name=node1_name)
iface6 = node1.add_component(model='NIC_Basic', name='nic3').get_interfaces()[0]
iface6.set_mode('auto')
net3.add_interface(iface6)


### Submit the updates

In [ ]:
slice.submit()

## Verify connectivity between Node1 and Node3

In [ ]:
slice = fablib.get_slice(slice_name)
node1 = slice.get_node(name=node1_name)
node3 = slice.get_node(name=node3_name)

In [ ]:
node3_addr = node3.get_interface(network_name=network2_name).get_ip_addr()

stdout, stderr = node1.execute(f'ping -c 5 {node3_addr}')

## Verify connectivity between Node1 and Node4

In [ ]:
slice = fablib.get_slice(slice_name)
node1 = slice.get_node(name=node1_name)
node4 = slice.get_node(name=node4_name)

In [ ]:
node4_addr = node4.get_interface(network_name=network3_name).get_ip_addr()

stdout, stderr = node1.execute(f'ping -c 5 {node4_addr}')

## Modify Slice 
NOTE: Prior to requesting any updates, it is essential to obtain the latest slice topology using slice = `fablib.get_slice(slice_name)`.

- Remove NIC1 from Node1.
- Eliminate Node2 from the slice.
- Remove Network Net1 from the slice.

![Updated Topology](./modify-delete.png)

In [ ]:
## NOTE: Always get the latest slice topolgy before requesting any updates
slice = fablib.get_slice(slice_name)


# Removing NIC1 from Node1
node1 = slice.get_node(name=node1_name)
node1_nic1 = node1.get_component(name="nic1")
node1_nic1.delete()


# Removing Node2 from Slice
node2 = slice.get_node(name=node2_name)
node2.delete()


# Net1 is a wide area network and no longer have two participants after Node1 being removed
# Removing the network
net1 = slice.get_network(name=network1_name)
net1.delete()

### Submit the updates

In [ ]:
slice.submit()


## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice = fablib.get_slice(slice_name)
slice.delete()